<a href="https://colab.research.google.com/github/CC-Shivansh-Gupta/logistic-_reg/blob/main/ann_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [195]:
import pandas as pd
import numpy as np
import matplotlib as plt
import copy, math
import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import layers
#from tensorflow.keras.layers.experimental import preprocessing

data = pd.read_csv('updated_air_quality_data.csv')
data.columns = data.columns.str.replace(' ', '_')
data
#print(type(std_data))
#print(std_data.info())

,Unique_ID,Name,Measure,Geo_Type_Name,Geo_Place_Name,Time_Period,Start_Date,Data_Value,Air_Quality_Category
0,179772,Emissions,Density,UHF42,Queens,Other,2015-01-01,0.3,Good
1,179785,Emissions,Density,UHF42,Unknown,Other,2015-01-01,1.2,Good
2,178540,General Pollution,Miles,UHF42,Unknown,Annual Average,2011-12-01,8.6,Good
3,178561,General Pollution,Miles,UHF42,Queens,Annual Average,2011-12-01,8.0,Good
4,823217,General Pollution,Miles,UHF42,Queens,Summer,2022-06-01,6.1,Good
...,...,...,...,...,...,...,...,...,...
18011,816914,General Pollution,Miles,CD,Unknown,Other,2019-01-01,12.9,Moderate
18012,816913,General Pollution,Miles,CD,Unknown,Other,2010-01-01,14.7,Moderate
18013,816872,General Pollution,Miles,UHF42,Unknown,Other,2010-01-01,43.4,Bad
18014,816832,General Pollution,Miles,UHF42,Queens,Other,2010-01-01,65.8,Bad


In [196]:
rows = data.shape[0]
cols = data.shape[1]
print(rows, cols)

18016 9


In [197]:
data.Unique_ID.describe()
print(data.Unique_ID.unique().shape)

(18016,)


In [198]:
if data.Unique_ID.unique().shape[0] == rows:
 data.set_index('Unique_ID', inplace = True)
data

,Name,Measure,Geo_Type_Name,Geo_Place_Name,Time_Period,Start_Date,Data_Value,Air_Quality_Category
Unique_ID,,,,,,,,
179772,Emissions,Density,UHF42,Queens,Other,2015-01-01,0.3,Good
179785,Emissions,Density,UHF42,Unknown,Other,2015-01-01,1.2,Good
178540,General Pollution,Miles,UHF42,Unknown,Annual Average,2011-12-01,8.6,Good
178561,General Pollution,Miles,UHF42,Queens,Annual Average,2011-12-01,8.0,Good
823217,General Pollution,Miles,UHF42,Queens,Summer,2022-06-01,6.1,Good
...,...,...,...,...,...,...,...,...
816914,General Pollution,Miles,CD,Unknown,Other,2019-01-01,12.9,Moderate
816913,General Pollution,Miles,CD,Unknown,Other,2010-01-01,14.7,Moderate
816872,General Pollution,Miles,UHF42,Unknown,Other,2010-01-01,43.4,Bad


In [199]:
rows = data.shape[0]
cols = data.shape[1]
print(rows, cols)

18016 8


In [200]:
#think how to do it on your own
non_numeric_cols = data.select_dtypes(exclude=['number']).columns
cols_arr = np.where(data.columns.isin(non_numeric_cols))[0]
print(cols_arr)

drop_cols = list(data.columns)
drop_cols.pop(cols-1)
print(drop_cols)

'''for i in range(cols):
  data.column_arr[i].describe()'''

[0 1 2 3 4 5 7]
['Name', 'Measure', 'Geo_Type_Name', 'Geo_Place_Name', 'Time_Period', 'Start_Date', 'Data_Value']


'for i in range(cols):\n  data.column_arr[i].describe()'

In [201]:
data.Name.describe()
data.Data_Value.unique()

array([  0.3,   1.2,   8.6, ...,  95.2,  62.3, 123.9])

In [202]:
data = data.replace('Unknown', '0')
data

,Name,Measure,Geo_Type_Name,Geo_Place_Name,Time_Period,Start_Date,Data_Value,Air_Quality_Category
Unique_ID,,,,,,,,
179772,Emissions,Density,UHF42,Queens,Other,2015-01-01,0.3,Good
179785,Emissions,Density,UHF42,0,Other,2015-01-01,1.2,Good
178540,General Pollution,Miles,UHF42,0,Annual Average,2011-12-01,8.6,Good
178561,General Pollution,Miles,UHF42,Queens,Annual Average,2011-12-01,8.0,Good
823217,General Pollution,Miles,UHF42,Queens,Summer,2022-06-01,6.1,Good
...,...,...,...,...,...,...,...,...
816914,General Pollution,Miles,CD,0,Other,2019-01-01,12.9,Moderate
816913,General Pollution,Miles,CD,0,Other,2010-01-01,14.7,Moderate
816872,General Pollution,Miles,UHF42,0,Other,2010-01-01,43.4,Bad


In [203]:
def extraction(column):
  name_arr1 = np.empty(rows, dtype = object)
  for j in range(rows):
    name_arr1[j] = data.iloc[j, column]
  #print(name_arr1)
  unique_names1 = np.unique(name_arr1)
  return name_arr1, unique_names1

In [204]:
def one_hot_label(arr, unique):
  arrofarr_list = []
  for j in range(len(unique)):
    arr_name = np.empty(len(arr), dtype = object)
    for i in range(len(arr)):
      arr_name[i] = arr[i]
    arrofarr_list.append(arr_name)

  arrofarr_tup = tuple(arrofarr_list)
  arrofarr = np.row_stack(arrofarr_tup)
  for j in range(len(unique)):
    for i in range(len(arr)):
      if arrofarr[j][i] == unique[j]:
        arrofarr[j][i] = '1'
      else:
        arrofarr[j][i] = '0'
  #print(arrofarr)
  return arrofarr

In [205]:
arr_arr_list = []
un_arr_list = []

for j in range(len(cols_arr)-1):
  name_arr, unique_names = extraction(cols_arr[j])
  arr_arr_list.append(name_arr)
  un_arr_list.append(unique_names)

'''arr_arr_tup = tuple(arr_arr_list)
arr_arr = np.row_stack(arr_arr_tup)
un_arr_tup = tuple(un_arr_list)
un_arr = np.row_stack(un_arr_tup)'''
#print(arr_arr_list)
#print(un_arr_list)

'arr_arr_tup = tuple(arr_arr_list)\narr_arr = np.row_stack(arr_arr_tup)\nun_arr_tup = tuple(un_arr_list)\nun_arr = np.row_stack(un_arr_tup)'

In [206]:
for j in range(len(cols_arr)-1):
  arrayofarr = one_hot_label(arr_arr_list[j], un_arr_list[j])
  for i in range(len(un_arr_list[j])):
    data.loc[:, un_arr_list[j][i]] = arrayofarr[i]
data

,Name,Measure,Geo_Type_Name,Geo_Place_Name,Time_Period,Start_Date,Data_Value,Air_Quality_Category,Asthma-Related,Emissions,...,2019-06-01,2019-12-01,2020-01-01,2020-06-01,2020-12-01,2021-01-01,2021-06-01,2021-12-01,2022-01-01,2022-06-01
Unique_ID,,,,,,,,,,,,,,,,,,,,,
179772,Emissions,Density,UHF42,Queens,Other,2015-01-01,0.3,Good,0,1,...,0,0,0,0,0,0,0,0,0,0
179785,Emissions,Density,UHF42,0,Other,2015-01-01,1.2,Good,0,1,...,0,0,0,0,0,0,0,0,0,0
178540,General Pollution,Miles,UHF42,0,Annual Average,2011-12-01,8.6,Good,0,0,...,0,0,0,0,0,0,0,0,0,0
178561,General Pollution,Miles,UHF42,Queens,Annual Average,2011-12-01,8.0,Good,0,0,...,0,0,0,0,0,0,0,0,0,0
823217,General Pollution,Miles,UHF42,Queens,Summer,2022-06-01,6.1,Good,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816914,General Pollution,Miles,CD,0,Other,2019-01-01,12.9,Moderate,0,0,...,0,0,0,0,0,0,0,0,0,0
816913,General Pollution,Miles,CD,0,Other,2010-01-01,14.7,Moderate,0,0,...,0,0,0,0,0,0,0,0,0,0
816872,General Pollution,Miles,UHF42,0,Other,2010-01-01,43.4,Bad,0,0,...,0,0,0,0,0,0,0,0,0,0


In [207]:
for j in range(len(drop_cols)):
  data.drop(drop_cols[j], axis = 1, inplace = True)
data

,Air_Quality_Category,Asthma-Related,Emissions,General Pollution,Hospitalizations,Concentration,Density,Miles,Rates,Borough,...,2019-06-01,2019-12-01,2020-01-01,2020-06-01,2020-12-01,2021-01-01,2021-06-01,2021-12-01,2022-01-01,2022-06-01
Unique_ID,,,,,,,,,,,,,,,,,,,,,
179772,Good,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179785,Good,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
178540,Good,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
178561,Good,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
823217,Good,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816914,Moderate,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
816913,Moderate,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
816872,Bad,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [212]:
data = data.replace('Bad', '0', regex = True)
data = data.replace('Good', '2', regex = True)
data = data.replace('Moderate', '1', regex = True)
data

,Air_Quality_Category,Asthma-Related,Emissions,General Pollution,Hospitalizations,Concentration,Density,Miles,Rates,Borough,...,2019-06-01,2019-12-01,2020-01-01,2020-06-01,2020-12-01,2021-01-01,2021-06-01,2021-12-01,2022-01-01,2022-06-01
Unique_ID,,,,,,,,,,,,,,,,,,,,,
179772,2,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179785,2,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
178540,2,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
178561,2,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
823217,2,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
816914,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
816913,1,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
816872,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
'''def label(arrey, ooniq):
  for j in range(len(ooniq)):
    for i in range(len(arrey)):
      if arrey[i] == ooniq[j]:
        arrey[i] = j
  return arrey'''

In [208]:
#data.rename(columns={'Name': 'Name_Emissions'}, inplace = True)
#data['Name_General'] = data.loc[:, 'Name_Emissions']
#data
#data['Name_Emissions'] = data['Name_Emissions'].str.replace('Emissions', '1')
#data
#data.loc[:, 'Name_Hospi'] = name_arrofarr[3]
#data
'''for i in range(len(unique_names)):
  data.loc[:, unique_names[i]] = name_arrofarr[i]
data'''

'for i in range(len(unique_names)):\n  data.loc[:, unique_names[i]] = name_arrofarr[i]\ndata'

In [209]:
'''name_arr = np.empty(rows, dtype = object)
for j in range(rows):
  name_arr[j] = data.loc[j, 'Name']
print(name_arr)
unique_names = np.unique(name_arr)
#unique_names'''

"name_arr = np.empty(rows, dtype = object)\nfor j in range(rows):\n  name_arr[j] = data.loc[j, 'Name']\nprint(name_arr)\nunique_names = np.unique(name_arr)\n#unique_names"